In [4]:
import os
import requests
import json
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv

url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"

params = {
    "goodsGenre": "ALL",
    "goodsRegion": "ALL",
    "offset": 0,
    "pageSize": 50,
    "sorting": "OPEN_ASC"
}

headers = {
    "host": "tickets.interpark.com",
    "sec-ch-ua-platform": "Windows",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "accept": "application/json, text/plain, */*",
    "sec-ch-ua": "\"Chromium\";v=\"134\", \"Not:A-Brand\";v=\"24\", \"Google Chrome\";v=\"134\"",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-site": "same-origin",
    "sec-fetch-mode": "cors",
    "sec-fetch-dest": "empty",
    "referer": "https://tickets.interpark.com/contents/notice",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
}

try:
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    
    today_date = datetime.now().strftime('%Y년 %m월 %d일')
    tomorrow = (datetime.now() + timedelta(days=1)).date()
    
    print(f"=== {today_date} 티켓 오픈 정보 ===")
    message = f"<b>🎫 {today_date} 티켓 오픈 정보 🎫</b>\n\n"
    
    for ticket in data:
        open_time = ticket['openDateStr'][11:16]
        title = ticket['title']
        if len(title) > 40:
            title = title[:40] + "..."
        view_count = ticket['viewCount']
        goods_code = ticket['goodsCode']
        open_type = ticket['openTypeStr']
        
        # 날짜 문자열에서 연도, 월, 일 추출
        open_date_str = ticket['openDateStr'][:10]
        year, month, day = map(int, open_date_str.split('-'))
        ticket_date = datetime(year, month, day).date()
        
        # 날짜별 구분 및 서식 추가
        today = datetime.now().date()
        # 내일 날짜인지 확인 (오늘과 내일 티켓만 표시)
        if ticket_date > today + timedelta(days=1):
            break
        if ticket_date == today:
            date_emoji = "🔴 오늘"
        elif ticket_date == today + timedelta(days=1):
            date_emoji = "🟠 내일"
        else:
            date_emoji = f"⚪ {month}월 {day}일"
        
        # 각 티켓 정보를 깔끔하게 포맷팅
        message += f"<b>{date_emoji} [{open_time}]</b>\n"
        message += f"<b>{title}</b>\n"
        message += f"👁 조회수: {view_count}  |  🎟 예매코드: <code>{goods_code}</code>  |  📌{open_type}\n"
        message += "───────────────────\n"
        print(message)
except Exception as e:
    print(f"오류 발생: {e}")








=== 2025년 06월 09일 티켓 오픈 정보 ===
<b>🎫 2025년 06월 09일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [10:00]</b>
<b>어린이 뮤지컬 바다탐험대 옥토넛 시즌3〈바다넘어 육지까지〉-대구 </b>
👁 조회수: 178  |  🎟 예매코드: <code>25008218</code>  |  📌일반예매
───────────────────

<b>🎫 2025년 06월 09일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [10:00]</b>
<b>어린이 뮤지컬 바다탐험대 옥토넛 시즌3〈바다넘어 육지까지〉-대구 </b>
👁 조회수: 178  |  🎟 예매코드: <code>25008218</code>  |  📌일반예매
───────────────────
<b>🟠 내일 [10:00]</b>
<b>김창완밴드 콘서트 - 청주 </b>
👁 조회수: 923  |  🎟 예매코드: <code>25008276</code>  |  📌일반예매
───────────────────

<b>🎫 2025년 06월 09일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [10:00]</b>
<b>어린이 뮤지컬 바다탐험대 옥토넛 시즌3〈바다넘어 육지까지〉-대구 </b>
👁 조회수: 178  |  🎟 예매코드: <code>25008218</code>  |  📌일반예매
───────────────────
<b>🟠 내일 [10:00]</b>
<b>김창완밴드 콘서트 - 청주 </b>
👁 조회수: 923  |  🎟 예매코드: <code>25008276</code>  |  📌일반예매
───────────────────
<b>🟠 내일 [10:00]</b>
<b>가족뮤지컬 〈에그박사 - 해적 보물섬의 비밀〉 - 대전 </b>
👁 조회수: 218  |  🎟 예매코드: <code>25008153</code>  |  📌일반예매
───────────────────

<b>🎫 2025년 06월 09일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [10:00]</b>
<b>어린이 뮤지컬 바다탐험대 옥토